In [4]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf

# SET WORK DERC

# For my Mac
os.chdir('//Users//patricklee//Desktop//파이썬//DACON//AI_workout')

# For my Desktop
# os.chdir('C://Users//BIS_COM//data//dacon/workout/')

In [5]:
train=pd.read_csv('./train_features.csv')
train_labels=pd.read_csv('./train_labels.csv')
test=pd.read_csv('./test_features.csv')
submission=pd.read_csv('./sample_submission.csv')

In [7]:
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [47]:
import math

In [85]:
x = float(train.loc[1,"acc_x"])
y = float(train.loc[1,"acc_y"])
z = float(train.loc[1,"acc_z"])

In [86]:
z

-0.1824437456652969

In [87]:
x

1.2876964792955858

In [107]:

angle_y = math.atan(x/math.sqrt((y**2)+(z**2)))*radian
angle_x = math.atan(y/math.sqrt((x**2)+(z**2)))*radian

In [108]:
print(angle_x, angle_y)

-8.698302422231919 78.15983501052303
